In [26]:
import lilac as ll
import spacy
from sklearn.preprocessing import normalize
import numpy as np

In [2]:
import functools
from typing import TYPE_CHECKING, Optional

if TYPE_CHECKING:
  from sentence_transformers import SentenceTransformer


@functools.cache
def _get_model(model_name: str, preferred_device: Optional[str]) -> 'SentenceTransformer':
  try:
    from sentence_transformers import SentenceTransformer
  except ImportError:
    raise ImportError('Could not import the "sentence_transformers" python package. '
                      'Please install it with `pip install "lilac[gte]".')
  return SentenceTransformer(model_name, device=preferred_device)


def get_model(model_name: str,
              optimal_batch_sizes: dict[str, int] = {}) -> tuple[int, 'SentenceTransformer']:
  """Get a transformer model and the optimal batch size for it."""
  try:
    import torch.backends.mps
  except ImportError:
    raise ImportError('Could not import the "sentence_transformers" python package. '
                      'Please install it with `pip install "lilac[gte]".')
  preferred_device: Optional[str] = None
  if torch.backends.mps.is_available():
    preferred_device = 'mps'
  elif not torch.backends.mps.is_built():
    print('MPS not available because the current PyTorch install was not built with MPS enabled.')

  batch_size = optimal_batch_sizes[preferred_device or '']
  return batch_size, _get_model(model_name, preferred_device)


In [8]:
batch_size, model = get_model(
  'thenlper/gte-small',
  {
    '': 64,  # Default batch size.
    'mps': 256,
  })

tokenizer = spacy.blank('en')
tokenizer.add_pipe('sentencizer')

In [105]:
text = """
The following phrase can be considered as mainly negative but with a touch of humor:

"rather like being trapped while some weird relative trots out the video he took of the family vacation to Stonehenge."

Here is a detailed analysis of the phrase:

1. Trapped: The feeling of being trapped evokes a negative emotion as it signifies a lack of freedom or the inability to escape a situation that is undesirable or unwanted. The mental state of feeling trapped can create a feeling of frustration, discomfort and anxiety.

2. Weird relative: The reference to a "weird relative" has a mildly negative connotation, as it implies that the relative has some peculiar, odd, or unusual traits that may be off-putting, or at least different from what is commonly accepted. It is important to note that "weird" does not always have to mean bad or negative, but it often highlights something that is not quite within the realm of predictability or normalcy.

3. Trots out the video: "Trots out" suggests that the relative is somewhat eager or enthusiastic about sharing the video, possibly without considering whether others in the audience share the same level of interest. This can be viewed as a somewhat negative or at least a bothersome experience for the listener or the audience mentioned in the phrase.

4. Family vacation to Stonehenge: The content of the video, a family vacation to Stonehenge, could be considered neutral or even positive (as vacations often involve enjoyable experiences and memories). However, in the context of this phrase – primarily due to the trapped feeling and the weird relative – it becomes part of a negative overall experience.

5. The phrase as a whole presents a situation that most people would consider to be negative due to the feeling of being trapped, the involvement of an offbeat relative, and the forced and possibly unwanted viewing of something of little or no interest to them.

With that said, there is a subtle humor in the phrase that may give it an overall light-heartedness, despite the negative connotations. The use of colorful language, a relatable scenario, and a lively description of the weird relative may make the reader or listener smirk or chuckle – even if the actual situation depicted is less than desirable. 
"""

p1 = True
CONTEXT_SENT_SIZE = 4
for doc in tokenizer.pipe([text]):
  sents = list(doc.sents)

  similaries_to_previous = [1.0] * CONTEXT_SENT_SIZE
  for i in range(CONTEXT_SENT_SIZE, len(sents)):
    sent = sents[i]
    context = ' '.join([str(sents[j]) for j in range(i - CONTEXT_SENT_SIZE, i)])
    context_emb = normalize(np.expand_dims(np.array(model.encode(context)), 0))

    sent_emb = normalize(np.expand_dims(np.array(model.encode(sent.text)), 0))

    similarity = np.dot(context_emb, np.squeeze(sent_emb))[0]
    similaries_to_previous.append(similarity)
    if p1:
      print('----------------------------------------------------------------')
      print('[Context]:')
      print(context)
      print('----------------')
      print('[Sentence]:')
      print(sent)
      print('----------------')
      print('\n[Dot product]: ', similarity)
      print('----------------------------------------------------------------')

  print(similaries_to_previous)
  last_id = 0
  for i, sim in enumerate(similaries_to_previous):
    if sim < 0.8:
      chunk = ''.join([str(sents[j]) for j in range(last_id, i)])
      print(f'-----------------------------[ {sim} ]-----------------------------------')
      print(chunk)
      last_id = i

  chunk = ''.join([str(sents[j]) for j in range(last_id, len(sents))])
  print(chunk)


----------------------------------------------------------------
[Context]:

The following phrase can be considered as mainly negative but with a touch of humor:

"rather like being trapped while some weird relative trots out the video he took of the family vacation to Stonehenge." 

Here is a detailed analysis of the phrase:

1. Trapped: The feeling of being trapped evokes a negative emotion as it signifies a lack of freedom or the inability to escape a situation that is undesirable or unwanted. The mental state of feeling trapped can create a feeling of frustration, discomfort and anxiety.
----------------
[Sentence]:


2.
----------------

[Dot product]:  0.7707396
----------------------------------------------------------------
----------------------------------------------------------------
[Context]:


Here is a detailed analysis of the phrase:

1. Trapped: The feeling of being trapped evokes a negative emotion as it signifies a lack of freedom or the inability to escape a situat